In [6]:
import os
def parse_filename(file_path):

    filename = os.path.basename(file_path)
    # Split the filename using the double underscore separator
    parts = filename.split('__')
    
    # Check if the filename is in the correct format
    if len(parts) < 3:
        raise ValueError("Filename format is incorrect. Expected format: 'product__source__fname'")
    
    # Extract the product and source values
    product = parts[0]
    source = parts[1]
    
    return product, source

In [11]:
# Cloud Function entry point
import os
import pandas as pd
from authentification import access_secret_version, download_file_from_bucket, execute_postgres_query, init_vertex_ai
from  Functions.survey_problem_solution import process_survey_data
from Functions.five_Star_Review import process_five_star_reviews
from Functions.social_media import process_social_media_data
from Functions.voice_call import process_voice_call_data

##event and context is passed by google bucket
def process_data(event,context):
    init_vertex_ai()

    # project_id = '903333611831'
    
    # db_user_secret_id = 'DB_USER'
    # db_password_secret_id = 'DB_PASS'
    # storage_bucket_name = 'jbaaam_upload'
    
    # # Retrieve secrets
    # db_user = access_secret_version(db_user_secret_id, project_id)
    # db_password = access_secret_version(db_password_secret_id, project_id)

    # ##Get info from trigger in bucket
    # bucket_name = event['bucket']
    # file_name = event['name']
    
    # Download the file from Google Cloud Storage
    # local_file_path = download_file_from_bucket(storage_bucket_name, file_name)
    local_file_path='/Users/phonavitra/Desktop/term 5/Service Studio/Test/Others__Social Media__Cloud_function_test.csv'
    product, source=parse_filename(local_file_path)

    # df = pd.read_csv(local_file_path)

    if source == "Product Survey" or source == "Problem Solution Survey":
        data= process_survey_data(product, source, local_file_path)
    elif source == "Call Center":
        data=process_voice_call_data(local_file_path, product,source)
    elif source == "Social Media":
        data= process_social_media_data(local_file_path,product,source)
    elif source == "5 Star Review":
        data= process_five_star_reviews(local_file_path,product,source)
    else:
        # Raise a ValueError for unsupported source values
        raise ValueError("Source not supported: {}".format(source))


    # Insert processed data into PostgreSQL
    # query = f"INSERT INTO your_table (column1, column2) VALUES ('value1', 'value2')"
    # execute_postgres_query(db_user, db_password, 'your_db_name', 'your_db_host', query)
    
    # # Delete the temporary file
    # os.remove(local_file_path)

    # return "Data processing and storage complete"
    output_file_path = f'/Users/phonavitra/Desktop/term 5/Service Studio/Test/Cloud_function_result.csv'
    try:
        data.to_csv(output_file_path, index=False)
        print(f"Data transformation complete. File saved to: {output_file_path}")
    except Exception as e:
        print(f"Error saving transformed data: {e}")
        raise




ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.